# Drug Consumption: Pré-processamento de Dados

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('data/drug_consumption.csv', index_col="ID")

# Pré-processamento dos dados
Os dados do dataset estão em valores pouco legíveis, assim, alguns processamentos foram executados para possibilitar o uso em classificadores.

## Idade - "Age"
Engloba um range de idades, entretanto, é remapeado (posteriormente) para valores discretos, mantendo a distância entre os itens.

In [3]:
age_col = {
    -0.95197: "18 - 24",
    -0.07854: "25 - 34",
    0.49788: "35 - 44",
    1.09449: "45 - 54",
    1.82213: "55 - 64",
    2.59171: "65+",
}

# Se for necessário, estão guardados valores nominais no dataset
data["Age_"] = data["Age"].replace(age_col)

## Gênero - "Gender"
Dividido entre "Female" e "Male". Aqui, é transformada de forma que a coluna indica se o indivíduo é homem ou não.
Poderia ser feito com OneHotEncoder, mas escolhemos poupar o uso de mais colunas.

In [4]:
# gender_col = {0.48246: "Female", -0.48246: "Male"}
gender_col = {0.48246: 0, -0.48246: 1}
data["Gender"] = data["Gender"].replace(gender_col)

## Escolaridade - "Education"
Indica o nível de escolaridade do indivíduo. Testamos colocar ordenado com OrdinalEncoder, da forma que a coluna é apresentada abaixo.

In [5]:
education_col = {
    -2.43591: "Left School Before 16 years",
    -1.73790: "Left School at 16 years",
    -1.43719: "Left School at 17 years",
    -1.22751: "Left School at 18 years",
    -0.61113: "Some College, No Certificate Or Degree",
    -0.05921: "Professional Certificate/Diploma",
    0.45468: "University Degree",
    1.16365: "Masters Degree",
    1.98437: "Doctorate Degree",
}

data["Education_"] = data["Education"].replace(education_col)

## País - "Country"
Indica o país de origem do indivíduo. Será mais relevante na análise de distribuição dos dados, mas pode causar algum bias desnecessário na classificação.

In [6]:
country_col = {
    -0.09765: "Australia",
    0.24923: "Canada",
    -0.46841: "New Zealand",
    -0.28519: "Other",
    0.21128: "Republic of Ireland",
    0.96082: "UK",
    -0.57009: "USA",
}
data["Country"] = data["Country"].replace(country_col)

## Etnia - "Ethnicity"
Indica a etnia do indivíduo. Pode ser mais relevante na análise dos dados, mas parece irrelevante e enviezado utilizá-la no classificador.

In [7]:
ethnicity_col = {
    -0.50212: "Asian",
    -1.10702: "Black",
    1.90725: "Mixed-Black/Asian",
    0.12600: "Mixed-White/Asian",
    -0.22166: "Mixed-White/Black",
    0.11440: "Other",
    -0.31685: "White",
}
data["Ethnicity"] = data["Ethnicity"].replace(ethnicity_col)

## Colunas das substâncias
Existem 18 substâncias presentes no dataset. Os valores apresentados nas colunas indicam qual foi a última vez que a substância foi utilizada. Nesse caso, remapeamos cada valor para um domínio discreto em que a distância de tempo é mantida.

É um range de [0, 6] que representa, respectivamente:
- (0) "nunca";
- (1) "há mais de uma década";
- (2) "na última década";
- (3) "no último ano";
- (4) "no último mês";
- (5) "na última semana" e
- (6) "no dia anterior".

In [8]:
drugs = [
    "Alcohol",
    "Amphet",
    "Amyl",
    "Benzos",
    "Caff",
    "Cannabis",
    "Choc",
    "Coke",
    "Crack",
    "Ecstasy",
    "Heroin",
    "Ketamine",
    "Legalh",
    "LSD",
    "Meth",
    "Mushrooms",
    "Nicotine",
    "Semer",
    "VSA",
]

order_ = {
    "CL0": "Never Used",
    "CL1": "Used over a Decade Ago",
    "CL2": "Used in Last Decade",
    "CL3": "Used in Last Year",
    "CL4": "Used in Last Month",
    "CL5": "Used in Last Week",
    "CL6": "Used in Last Day",
}

# Deixamos os labels originais no dataset em colunas alternativas com underline no final (caso for necessário saber o label nominal para visualizações)
data[[x + "_" for x in drugs]] = data[drugs].applymap(lambda x: order_[x])

## Medidas de Personalidade
O dataset ainda apresenta algumas métricas chamadas de "personality measurements". São elas:
- NEO-FFI-R (neuroticism, extraversion, openness to experience, agreeableness, and conscientiousness)
- BIS-11 (impulsivity)
- ImpSS (sensation seeking)

In [9]:
data.rename(
    columns={
        "Nscore": "Neuroticism",
        "Escore": "Extraversion",
        "Oscore": "Openness",
        "Ascore": "Agreeableness",
        "Cscore": "Conscientiousness",
        "Impulsive": "Impulsiveness",
        "SS": "Sensationness",
    },
    inplace=True,
)

## Aplicação de OrdinalEncoder e OneHotEncoder
Aplicamos OrdinalEncoder em colunas como "Age" e as colunas de substâncias, que apresentam valores com ordem explícita e OneHotEncoder para características categóricas sem senso perceptível de distância.

In [10]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer


ordCols = ["Age", "Education"] + drugs
# catCols = ["Ethnicity"]

preprocessor = ColumnTransformer(
    [
        ("ordinal", OrdinalEncoder(dtype=int), ordCols),
        # ("categorical", OneHotEncoder(sparse_output=False), catCols),
    ],
    verbose_feature_names_out=False,
    remainder="passthrough",
)
preprocessor.set_output(transform="pandas")

df_out : pd.DataFrame = preprocessor.fit_transform(data)

/home/ngmonteiro/miniconda3/envs/py/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [11]:
df_out.to_csv("data/drug_consumption_preprocessed.csv")